In [42]:
# Importation des bibliothèques

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

In [43]:
# Chargement des données
df = pd.read_excel('data/Base_ISM_PARIS_301224.xlsx')

In [ ]:
# Afficher les premières lignes pour vérifie
df.head()

# Exploration de données

In [ ]:
# Information générale
print(df.info())
print(df.describe(include='all'))


In [ ]:
# Vérification des valeurs manquantes
missing_values = df.isnull().sum()

missing_values = missing_values[missing_values > 0]
print(missing_values)

In [ ]:
df['f0'] = df['f0'].fillna(0)
df['bien'] = df['bien'].fillna(df['bien'].median())
df['education'] = df['education'].fillna('non renseigné')
df['sectinst'] = df['sectinst'].fillna('non renseigné')
df['secteur'] = df['secteur'].fillna('non renseigné')
df['popemp'] = df['popemp'].fillna('non renseigné')
df['ep'] = df['ep'].fillna('non renseigné')
df['depalim'] = df['depalim'].fillna(df['depalim'].median())
df['cq25'] = df['cq25'].fillna(df['cq25'].mode()[0]) 

# Vérification après traitement des valeurs manquantes
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]
print(missing_values)

In [ ]:
# Conversion des colonnes float en int
df[df.select_dtypes('float').columns] = df.select_dtypes('float').astype(int)
print(df.dtypes)

## Visualisation

In [ ]:
# Distribution de l'âge
plt.figure(figsize=(8, 5))
sns.histplot(df['age'], kde=True)
plt.title("Distribution de l'âge")
plt.show()

In [ ]:
# Distribution de la Taille du Ménage
plt.style.use("seaborn-v0_8-whitegrid")
plt.figure(figsize=(10, 6))
sorted_hhsize = sorted(df['hhsize'].unique())
ax = sns.countplot(data=df, x='hhsize', order=sorted_hhsize, color="#4C72B0")
for p in ax.patches:
    height = p.get_height()
    ax.annotate(f'{int(height)}', (p.get_x() + p.get_width() / 2, height), ha='center', va='bottom', fontsize=10, color='black', fontweight='bold', xytext=(0, 5), textcoords='offset points')
plt.title("Distribution de la Taille du Ménage", fontsize=16, fontweight="bold", pad=15)
plt.xlabel("Taille du ménage (hhsize)", fontsize=14, labelpad=10)
plt.ylabel("Nombre d'observations", fontsize=14, labelpad=10)
plt.xticks(rotation=0, fontsize=12)
plt.yticks(fontsize=12)
sns.despine(left=True)
plt.show()

In [ ]:
# Relation entre deux variables( depalim vs bien)
plt.figure(figsize=(8, 5))
sns.scatterplot(x='depalim', y='bien', data=df)
plt.title("Relation entre la dépense alimentaire et les biens")
plt.show()

In [ ]:
# Répartition des milieux de résidence
plt.figure(figsize=(8, 5))
sns.countplot(x='milieu', data=df)
plt.title("Répartition des milieux de résidence")
plt.show()

In [ ]:
# Matrice de corrélation pour les variables numériques
numeric_vars = ['age', 'hhsize', 'depalim', 'depenseM', 'bien']
corr_matrix = df[numeric_vars].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matrice de Corrélation des Variables Numériques")
plt.show()


In [ ]:
categorical_vars = ['milieu', 'strate', 'b05_region', 'P0']
plt.figure(figsize=(18, 12))

for i, var in enumerate(categorical_vars, 1):
    plt.subplot(2, 2, i)
    
    sns.countplot(data=df, x=var, hue=var, palette="viridis", legend=False)
    plt.title(f"Répartition de {var}", fontsize=14)
    plt.xticks(rotation=45, fontsize=10)
    plt.yticks(fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Relations entre variables (Dépense Totale du Ménage par Niveau de Pauvreté)
plt.figure(figsize=(10, 6))

sns.boxplot(x='P0', y='depenseM', data=df, hue='P0', palette="Set2", dodge=False, legend=False)

plt.title("Dépense Totale du Ménage par Niveau de Pauvreté (P0)", fontsize=14)
plt.xlabel("Niveau de Pauvreté", fontsize=12)
plt.ylabel("Dépense Totale du Ménage", fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

In [ ]:
# Analyse bivariée avec des Variables Numériques
sns.pairplot(df[numeric_vars], diag_kind="kde", plot_kws={'alpha': 0.6})
plt.suptitle("Pairplot des Variables Numériques", y=1.02)
plt.show()


In [ ]:
# Identifier les colonnes non numériques
categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()

binary_columns = [col for col in categorical_columns if df[col].nunique(dropna=True) == 2]
multi_columns = [col for col in categorical_columns if df[col].nunique(dropna=True) > 2]

print("Colonnes binaires sélectionnées :", binary_columns)
print("Colonnes catégorielles multinomiales sélectionnées :", multi_columns)

In [ ]:
# Encodage
# 1. Label Encoding pour les variables binaires
label_encoder = LabelEncoder()
for col in binary_columns:
    df[col] = label_encoder.fit_transform(df[col])

# 2. One-Hot Encoding pour les variables à plusieurs catégories
df_encoded = pd.get_dummies(df, columns=multi_columns, drop_first=True)
print(df_encoded.head())

In [ ]:
from sklearn.preprocessing import StandardScaler

# Normalisation
# Identifier les colonnes numériques dans le DataFrame encodé
numeric_columns_encoded = [col for col in df_encoded.columns if col != 'P0' and np.issubdtype(df_encoded[col].dtype, np.number)]

# Appliquer la normalisation uniquement aux colonnes numériques
if numeric_columns_encoded:
    scaler = StandardScaler()
    df_encoded[numeric_columns_encoded] = scaler.fit_transform(df_encoded[numeric_columns_encoded])

print("Données après normalisation :")
print(df_encoded.head())


In [ ]:
# Vérifier que P0 est toujours discrète
print("Types de données après normalisation :")
print(df_encoded.dtypes)

# Vérifier les valeurs uniques de P0
print("Valeurs uniques de P0 :", df_encoded['P0'].unique())

In [ ]:
# Distribution de la variable cible
class_counts = df['P0'].value_counts(normalize=True)

plt.figure(figsize=(8, 6))

plt.pie(class_counts, labels=[f"Non Pauvre ({class_counts[0]*100:.2f}%)", f"Pauvre ({class_counts[1]*100:.2f}%)"], 
        autopct='%1.1f%%', startangle=90, colors=["#66b3ff", "#ff9999"], explode=(0.1, 0))  

plt.title('Distribution de la variable cible P0 (Niveau de pauvreté)', fontsize=15)

plt.axis('equal')  
plt.show()

print("Proportions des classes dans P0 :")
print(round(class_counts*100,2))

In [60]:

from imblearn.over_sampling import SMOTE

X = df_encoded.drop(columns=["P0"])
y = df_encoded["P0"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Application de SMOTE pour équilibrer les classes dans l'ensemble d'entraînement
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)


class_counts_smote = y_train_res.value_counts()

In [ ]:
# Distribution de la variable cible
plt.figure(figsize=(8, 6))


plt.pie(class_counts_smote, labels=[f"Non Pauvre ({class_counts_smote[0]*100/len(y_train_res):.2f}%)", f"Pauvre ({class_counts_smote[1]*100/len(y_train_res):.2f}%)"], 
        autopct='%1.1f%%', startangle=90, colors=["#66b3ff", "#ff9999"], explode=(0.1, 0)) 

plt.title('Distribution de la variable cible P0 après équilibrage avec SMOTE', fontsize=15)

plt.axis('equal') 
plt.show()

# Afficher les proportions des classes dans l'ensemble d'entraînement après SMOTE
print("Proportions des classes dans P0 après SMOTE :")
print(class_counts_smote / len(y_train_res))  # Calcul des proportions

In [62]:
# Initialisation des modèles avec des paramètres adaptés
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, solver='liblinear'),  # Augmenter max_iter et changer le solver
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(probability=True)
}

In [63]:
# Stocker les résultats des performances
results = {}
best_model = None
best_f1_score = 0

In [ ]:
# Entraînement et évaluation de chaque modèle
for model_name, model in models.items():
    print(f"Entraînement du modèle : {model_name}")
    
    try:
        # Entraîner le modèle
        model.fit(X_train_res, y_train_res)
        
        # Faire des prédictions
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None
        
        # Calculer les métriques
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_proba) if y_proba is not None else "N/A"
        
        # Stocker les résultats
        results[model_name] = {
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1 Score": f1,
            "ROC AUC": roc_auc
        }
        
        # Sélectionner le meilleur modèle basé sur le F1 Score
        if f1 > best_f1_score:
            best_f1_score = f1
            best_model = model_name
            best_model_instance = model
    
    except Exception as e:
        print(f"Erreur lors de l'entraînement du modèle {model_name} : {e}")

In [ ]:
# Afficher les résultats
results_df = pd.DataFrame(results).T
print("Résultats des modèles :")
print(results_df)

print(f"\nMeilleur modèle : {best_model} (F1 Score : {best_f1_score})")

In [66]:
# Réoptimisation des hyperparamètres pour le meilleur modèle
if best_model == "Logistic Regression":
    param_grid = {
        'C': [0.1, 1, 10, 100],
        'solver': ['liblinear', 'saga']
    }
elif best_model == "Decision Tree":
    param_grid = {
        'max_depth': [5, 10, 15, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
elif best_model == "Random Forest":
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [5, 10, None],
        'min_samples_split': [2, 5]
    }
elif best_model == "SVM":
    param_grid = {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf'],
        'gamma': ['scale', 'auto']
    }

In [ ]:
from sklearn.model_selection import GridSearchCV

# Optimisation avec GridSearchCV (si applicable)
if param_grid: 
    print(f"Optimisation des hyperparamètres pour {best_model}...")
    grid_search = GridSearchCV(best_model_instance, param_grid, cv=5, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train_res, y_train_res)
    best_model_instance = grid_search.best_estimator_
    print(f"Meilleurs hyperparamètres : {grid_search.best_params_}")

In [ ]:
import numpy as np

# Sélection des variables importantes (si possible)
if isinstance(best_model_instance, (RandomForestClassifier, LogisticRegression)):
    if hasattr(best_model_instance, 'feature_importances_'):
        feature_importances = pd.DataFrame({
            'Feature': X.columns,
            'Importance': best_model_instance.feature_importances_
        }).sort_values(by='Importance', ascending=False)
    elif hasattr(best_model_instance, 'coef_'):
        feature_importances = pd.DataFrame({
            'Feature': X.columns,
            'Importance': np.abs(best_model_instance.coef_[0])
        }).sort_values(by='Importance', ascending=False)
    
    # Sélectionner les variables importantes (par exemple, Importance > moyenne)
    selected_features = feature_importances[feature_importances['Importance'] > feature_importances['Importance'].mean()]['Feature'].tolist()
    print(f"Variables retenues : {selected_features}")
else:
    # Si le modèle ne fournit pas d'importance des variables, garder toutes les variables
    selected_features = X.columns.tolist()
    print("Le modèle ne fournit pas d'importance des variables. Toutes les variables sont conservées.")

In [69]:
# Créer un pipeline final avec les variables sélectionnées
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel

pipeline_final = Pipeline(steps=[
    ('selector', SelectFromModel(best_model_instance, threshold="mean")) if len(selected_features) < len(X.columns) else ('passthrough', 'passthrough'),
    ('classifier', best_model_instance)
])

In [70]:
# Entraîner le pipeline final
if len(selected_features) < len(X.columns):
    pipeline_final.fit(X_train_res[selected_features], y_train_res)
else:
    pipeline_final.fit(X_train_res, y_train_res)

In [ ]:
# Évaluer le pipeline final
y_pred_final = pipeline_final.predict(X_test[selected_features] if len(selected_features) < len(X.columns) else X_test)
accuracy_final = accuracy_score(y_test, y_pred_final)
f1_final = f1_score(y_test, y_pred_final)

print(f"Performance du pipeline final - Accuracy : {accuracy_final}, F1 Score : {f1_final}")

In [ ]:
import os
import joblib

# Sauvegarder le modèle dans le dossier trained_model
model_path = os.path.join("trained_model", "final_model.pkl")
joblib.dump(pipeline_final, model_path)

print(f"Modèle final sauvegardé à : {model_path}")